In [17]:
import pandas as pd
from sqlalchemy import create_engine

In [18]:
# Tabla 1
school_url = 'https://s3.amazonaws.com/General_V88/boomyeah2015/codingdojo/curriculum/content/chapter/School_Details.csv'
school = pd.read_csv(school_url)

# Tabla 2
facility_url = 'https://s3.amazonaws.com/General_V88/boomyeah2015/codingdojo/curriculum/content/chapter/School_Facility_Details.csv'
facility = pd.read_csv(facility_url)

In [11]:
facility.head(2)

,EstablishmentCode,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,S1034,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,Not Available,0,Available,1,Available,Not Available,0,Not Available,Not Available
1,S0516,10TH MILE GOVERNMENT PRIMARY SCHOOL,Not Available,0,Not Available,0,Available,Available,1,Not Available,Not Available


In [12]:
school.head(2)

,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,ConstituencyName,...,EstablishedDate,DistrictHQDistance (in Km),CRCDistance (in Km),BACDistance (in Km),IsRecognized,RecognizedBy,ContactPersonName,LowestClass,HighestClass,BoardName
0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,...,28126,58.0,8.0,16.0,Recognized,HRDD,NaN,Class 1,Class 5,NaN
1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,...,15342,50.0,0.0,6.0,Recognized,HRDD,NaN,Class 1,Class 12,C.B.S.E.


In [19]:
# creamos instancia SQL
engine = create_engine('sqlite:///school.db', echo=True)
# Conexion SQL
conn = engine.connect()

In [22]:
# add the dataframe as a table in sqlite
sqlite_table = "school_details"
school.to_sql(sqlite_table, conn, if_exists='fail')

sqlite_table2 = "school_facility_details"
facility.to_sql(sqlite_table2, conn, if_exists='fail')

In [23]:
# 1. ¿Cuál es el BACDistance promedio (en km)? Renombra la columna resultante como "avg_bac_distancia."
sql = """
SELECT AVG("BACDistance (in Km)") AS avg_bac_distance
FROM school_details
"""

# SQL a pandas
df = pd.read_sql_query(sql, engine)
df.head()

2022-04-01 17:06:14,309 INFO sqlalchemy.engine.Engine 
SELECT AVG("BACDistance (in Km)") AS avg_bac_distance
FROM school_details

2022-04-01 17:06:14,311 INFO sqlalchemy.engine.Engine [raw sql] ()


,avg_bac_distance
0,12.681822


In [24]:
# 2. ¿Cuál es el BACDistance promedio (en km) según nivel educativo del colegio?  Ordena los resultados de promedio mayor a promedio menor.
sql = """
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
ORDER BY avg_bac_distance DESC
"""

# SQL a pandas
df = pd.read_sql_query(sql, engine)
df.head()

2022-04-01 17:06:28,261 INFO sqlalchemy.engine.Engine 
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
ORDER BY avg_bac_distance DESC

2022-04-01 17:06:28,263 INFO sqlalchemy.engine.Engine [raw sql] ()


,SchoolLevel,avg_bac_distance
0,JHS,13.692647
1,PS,12.830787
2,SS,12.630872
3,SSS,10.863095
4,LPS,5.368421


In [25]:
# 3. Repite la consulta anterior, pero solo muestra los resultados donde la distancia promedio sea menor a 10 km.
sql = """
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
HAVING avg_bac_distance >= 10
ORDER BY avg_bac_distance DESC
"""

# SQL a pandas
df = pd.read_sql_query(sql, engine)
df.head()

2022-04-01 17:06:31,362 INFO sqlalchemy.engine.Engine 
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
HAVING avg_bac_distance >= 10
ORDER BY avg_bac_distance DESC

2022-04-01 17:06:31,365 INFO sqlalchemy.engine.Engine [raw sql] ()


,SchoolLevel,avg_bac_distance
0,JHS,13.692647
1,PS,12.830787
2,SS,12.630872
3,SSS,10.863095


In [26]:
# 4. Une las dos tablas (detalles de colegios y detalles de instalaciones 
# de los colegios). Comienza mostrando todas las columnas, pero limita 
# las filas resultantes a solo 2.
sql = """
SELECT *
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
LIMIT 2
"""

# SQL a pandas
df = pd.read_sql_query(sql, engine)
df.head()

2022-04-01 17:06:34,001 INFO sqlalchemy.engine.Engine 
SELECT *
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
LIMIT 2

2022-04-01 17:06:34,004 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,...,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,...,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Available,100,Available,2,Available,Available,1,Not Available,Not Available
1,1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,...,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Available,0,Available,5,Available,Available,2,Not Available,Available


In [27]:
# 5. Ahora, solo selecciona colegios donde el nivel de este sea Escuela Primaria y que no tengan biblioteca.
sql = """
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
"""

# SQL a pandas
df = pd.read_sql_query(sql, engine)
df.head()

2022-04-01 17:06:37,201 INFO sqlalchemy.engine.Engine 
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'

2022-04-01 17:06:37,204 INFO sqlalchemy.engine.Engine [raw sql] ()


,EstablishmentName,SchoolLevel,IsLibraryAvailable
0,EASWARAMMA SAI GURUKOOL ACCADEMY,PS,Not Available
1,NEW GARDEN BOARDING SCHOOL (SAJONG),PS,Not Available
2,"SARASWATI SHISHU VIDHYALAYA, BUDANG",PS,Not Available
3,SAWNEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,UPPER PACHAK GOVERNMENT PRIMARY SCHOOL,PS,Not Available


In [28]:
# 6. Toma la consulta anterior, pero ordena los resultados en orden alfabético por nombre.
sql = """
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
ORDER BY s.EstablishmentName
"""

# SQL a pandas
df = pd.read_sql_query(sql, engine)
df.head()

2022-04-01 17:06:41,349 INFO sqlalchemy.engine.Engine 
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
ORDER BY s.EstablishmentName

2022-04-01 17:06:41,351 INFO sqlalchemy.engine.Engine [raw sql] ()


,EstablishmentName,SchoolLevel,IsLibraryAvailable
0,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,PS,Not Available
1,10TH MILE GOVERNMENT PRIMARY SCHOOL,PS,Not Available
2,AAPGAZI GOVERNMENT PRIMARY SCHOOL,PS,Not Available
3,AARUBOTEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,AHO-YANGTAM GOVERNMENT PRIMARY SCHOOL,PS,Not Available


In [29]:
# Cerramos conexion
conn.close()